In [1]:
import akshare as ak
import pickle
import hashlib
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import os

In [2]:
def cached_Ashare_index_hist(symbol,start_date,end_date):
    _CACHE_DIR = "./ak_cache"
    os.makedirs(_CACHE_DIR, exist_ok=True)
    key = f"{symbol}_{start_date}_{end_date}"
    h = hashlib.md5(key.encode()).hexdigest()
    cache_file = os.path.join(_CACHE_DIR, f"{h}.pkl")
    if os.path.exists(cache_file):
        return pd.read_pickle(cache_file)
#  A股股票指数数据
    df = ak.index_zh_a_hist(symbol=symbol,
                            period="daily",
                           start_date=start_date,
                           end_date=end_date)
    # 避免并发写
    pd.to_pickle(df, cache_file + ".tmp")
    os.replace(cache_file + ".tmp", cache_file)
    time.sleep(0.3)          # 降频
    return df

def cached_Hshare_index_hist(symbol):
    _CACHE_DIR = "./ak_cache"
    os.makedirs(_CACHE_DIR, exist_ok=True)
    key = f"{symbol}"
    h = hashlib.md5(key.encode()).hexdigest()
    cache_file = os.path.join(_CACHE_DIR, f"{h}.pkl")
    if os.path.exists(cache_file):
        return pd.read_pickle(cache_file)
#  H股股票指数数据
    df =ak.stock_hk_index_daily_em(symbol=symbol)
    # 避免并发写
    pd.to_pickle(df, cache_file + ".tmp")
    os.replace(cache_file + ".tmp", cache_file)
    time.sleep(0.3)          # 降频
    return df

def cached_USshare_index_hist(symbol):
    _CACHE_DIR = "./ak_cache"
    os.makedirs(_CACHE_DIR, exist_ok=True)
    key = f"{symbol}"
    h = hashlib.md5(key.encode()).hexdigest()
    cache_file = os.path.join(_CACHE_DIR, f"{h}.pkl")
    if os.path.exists(cache_file):
        return pd.read_pickle(cache_file)
#  H股股票指数数据
    df =ak.index_us_stock_sina(symbol=symbol)
    # 避免并发写
    pd.to_pickle(df, cache_file + ".tmp")
    os.replace(cache_file + ".tmp", cache_file)
    time.sleep(0.3)          # 降频
    return df

def cached_bond_index_hist(symbol):
    _CACHE_DIR = "./ak_cache"
    os.makedirs(_CACHE_DIR, exist_ok=True)
    key = f"{symbol}"
    h = hashlib.md5(key.encode()).hexdigest()
    cache_file = os.path.join(_CACHE_DIR, f"{h}.pkl")
    if os.path.exists(cache_file):
        return pd.read_pickle(cache_file)
#  H股股票指数数据
    df =ak.bond_new_composite_index_cbond(indicator=symbol, period="总值")
    # 避免并发写
    pd.to_pickle(df, cache_file + ".tmp")
    os.replace(cache_file + ".tmp", cache_file)
    time.sleep(0.3)          # 降频
    return df

def cached_gold_index_hist(symbol):
    _CACHE_DIR = "./ak_cache"
    os.makedirs(_CACHE_DIR, exist_ok=True)
    key = f"{symbol}"
    h = hashlib.md5(key.encode()).hexdigest()
    cache_file = os.path.join(_CACHE_DIR, f"{h}.pkl")
    if os.path.exists(cache_file):
        return pd.read_pickle(cache_file)
#  H股股票指数数据
    df =ak.spot_hist_sge(symbol=symbol)
    # 避免并发写
    pd.to_pickle(df, cache_file + ".tmp")
    os.replace(cache_file + ".tmp", cache_file)
    time.sleep(0.3)          # 降频
    return df

In [15]:
import akshare as ak
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import os

# 设置中文显示
plt.rcParams["font.family"] = ["FangSong"]
sns.set(font_scale=1.2)
sns.set_style("whitegrid")


class AssetAllocationTracker:
    def __init__(self):
        # 定义需要跟踪的大类资产及其对应的指数代码
        self.asset_config = {
            # 股票类
            "hs300": {"index_code": "000300", "source": "stock", "frequency": "daily"},
            "zz500": {"index_code": "000905", "source": "stock", "frequency": "daily"},
            # "股票-恒生指数": {"index_code": "HSI", "source": "hk_stock", "frequency": "daily"},
            "spx500": {"index_code": ".INX", "source": "us_stock", "frequency": "daily"},
            
            # 债券类
            # "债券-国债10年": {"index_code": "000012", "source": "bond", "frequency": "daily"},
            "composite_bond_index": {"index_code": "财富", "source": "bond_index", "frequency": "daily"},
            
            # 商品类
            "comodity_gold": {"index_code": "Au99.99", "source": "commodity_gold", "frequency": "daily"},
            # "商品-原油": {"index_code": "CL", "source": "commodity_oil", "frequency": "daily"},
            # "商品-豆粕": {"index_code": "M", "source": "commodity_future", "frequency": "daily"},
            
            # # 现金类
            # "现金-货币基金": {"index_code": "H11025", "source": "fund", "frequency": "daily"},
        }
        
        # 存储获取的数据
        self.asset_data = {}
        self.returns_data = None
        
        # 结果存储路径
        self.result_dir = "asset_allocation_reports"
        if not os.path.exists(self.result_dir):
            os.makedirs(self.result_dir)

    
    
    def fetch_asset_data(self, days=365*3):
        """获取指定天数内的资产数据"""
        end_date = datetime.now().strftime("%Y%m%d")
        start_date = (datetime.now() - timedelta(days=days)).strftime("%Y%m%d")
        
        print(f"正在获取从 {start_date} 到 {end_date} 的资产数据...")
        
        for asset_name, config in self.asset_config.items():
            try:
                if config["source"] == "stock":
                    # A股指数
                    df = cached_Ashare_index_hist(symbol=config["index_code"], start_date=start_date,
                           end_date=end_date)
                    df["date"] = pd.to_datetime(df["日期"])
                    df["close"]=df['收盘']
                    df = df[(df["date"] >= start_date) & (df["date"] <= end_date)]
                    self.asset_data[asset_name] = df[["date", "close"]].rename(columns={"close": asset_name})
                
                
                elif config["source"] == "us_stock":
                    # 美股指数
                    df = cached_USshare_index_hist(symbol=config["index_code"])
                    df["date"] = pd.to_datetime(df["date"])
                    df = df[(df["date"] >= start_date) & (df["date"] <= end_date)]
                    self.asset_data[asset_name] = df[["date", "close"]].rename(columns={"close": asset_name})
                
                elif config["source"] == "bond_index":
                    # 国债指数
                    df = cached_bond_index_hist(symbol=config["index_code"])
                    df["date"] = pd.to_datetime(df["date"])
                    df = df[(df["date"] >= start_date) & (df["date"] <= end_date)]
                    self.asset_data[asset_name] = df[["date", "value"]].rename(columns={"date": "date", "value": asset_name})
                
                
                elif config["source"] == "commodity_gold":
                    # 黄金
                    df = cached_gold_index_hist(symbol=config["index_code"])
                    df["date"] = pd.to_datetime(df["date"])
                    df = df[(df["date"] >= start_date) & (df["date"] <= end_date)]
                    self.asset_data[asset_name] = df[["date", "close"]].rename(columns={"date": "date", "close": asset_name})
                
 
                print(f"成功获取 {asset_name} 数据，共 {len(self.asset_data[asset_name])} 条记录")
            
            except Exception as e:
                print(f"获取 {asset_name} 数据失败: {str(e)}")
        
        # 合并所有资产数据
        self._merge_asset_data()
        
        # 计算收益率
        self._calculate_returns()
    
    def _merge_asset_data(self):
        """合并所有资产数据，按日期对齐"""
        if not self.asset_data:
            raise ValueError("没有可合并的资产数据，请先调用fetch_asset_data方法")
        
        # 以第一个资产的日期为基础进行合并
        merged_df = None
        for asset_name, df in self.asset_data.items():
            if merged_df is None:
                merged_df = df
            else:
                merged_df = pd.merge(merged_df, df, on="date", how="outer")
        
        # 按日期排序
        merged_df = merged_df.sort_values("date").reset_index(drop=True)
        
        # 填充缺失值（使用前向填充）
        merged_df = merged_df.fillna(method="ffill").dropna()
        
        self.merged_data = merged_df
        print(f"数据合并完成，共 {len(merged_df)} 个交易日")
    
    def _calculate_returns(self):
        """计算各类资产的收益率"""
        if self.merged_data is None:
            raise ValueError("没有合并的资产数据，请先调用fetch_asset_data方法")
        
        # 提取价格数据（排除date列）
        price_data = self.merged_data.drop(columns=["date"])
        
        # 计算日收益率
        self.returns_data = price_data.pct_change().dropna()
        print(f"收益率计算完成，共 {len(self.returns_data)} 个有效数据点")
    
    def calculate_volatility(self, window=20):
        """计算波动率（默认20个交易日窗口）"""
        if self.returns_data is None:
            raise ValueError("没有收益率数据，请先调用fetch_asset_data方法")
        
        # 计算滚动波动率（标准差）并年化
        volatility = self.returns_data.rolling(window=window).std() * np.sqrt(252)
        return volatility
    
    def calculate_correlation(self, window=60):
        """计算相关性（默认60个交易日窗口）"""
        if self.returns_data is None:
            raise ValueError("没有收益率数据，请先调用fetch_asset_data方法")
        
        # 计算滚动相关性
        correlations = {}
        for i in range(window, len(self.returns_data) + 1):
            date = self.returns_data.index[i-1]
            window_data = self.returns_data.iloc[i-window:i]
            correlations[date] = window_data.corr()
        
        return correlations
    
    def calculate_performance_metrics(self):
        """计算各类资产的关键绩效指标"""
        if self.returns_data is None:
            raise ValueError("没有收益率数据，请先调用fetch_asset_data方法")
        
        metrics = {}
        
        # 计算总收益率
        total_return = (1 + self.returns_data).prod() - 1
        
        # 计算年化收益率
        days = len(self.returns_data)
        annual_return = (1 + total_return) ** (252 / days) - 1
        
        # 计算年化波动率
        annual_volatility = self.returns_data.std() * np.sqrt(252)
        
        # 计算夏普比率（假设无风险利率为3%）
        risk_free_rate = 0.03
        sharpe_ratio = (annual_return - risk_free_rate) / annual_volatility
        
        # 计算最大回撤
        cumulative_returns = (1 + self.returns_data).cumprod()
        rolling_max = cumulative_returns.cummax()
        drawdown = (cumulative_returns - rolling_max) / rolling_max
        max_drawdown = drawdown.min()
        
        # 整理指标
        for asset in self.returns_data.columns:
            metrics[asset] = {
                "总收益率": total_return[asset],
                "年化收益率": annual_return[asset],
                "年化波动率": annual_volatility[asset],
                "夏普比率": sharpe_ratio[asset],
                "最大回撤": max_drawdown[asset]
            }
        
        return pd.DataFrame(metrics).T

    def generate_report(self):
            """生成资产配置分析报告，修复日期显示错误"""
            if self.returns_data is None:
                raise ValueError("没有收益率数据，请先调用fetch_asset_data方法")
            
            report_date = datetime.now().strftime("%Y%m%d")
            report_filename = f"{self.result_dir}/资产配置分析报告_{report_date}.html"
            
            # 计算各类指标
            performance_metrics = self.calculate_performance_metrics()
            volatility = self.calculate_volatility()
            
            # 修复波动率日期处理
            # 1. 确保波动率数据的索引是datetime格式
            if not pd.api.types.is_datetime64_any_dtype(volatility.index):
                # 尝试将索引从字符串转换为datetime
                try:
                    volatility.index = pd.to_datetime(volatility.index)
                except:
                    # 如果转换失败，使用合并数据中的最新日期
                    latest_date_from_merged = pd.to_datetime(self.merged_data['date'].iloc[-1])
                    volatility.index = pd.DatetimeIndex([latest_date_from_merged] * len(volatility))
            
            # 2. 获取最新波动率对应的日期
            latest_volatility = volatility.iloc[-1]
            latest_volatility_date = volatility.index[-1]
            
            # 如果日期仍是无效值（如Timestamp(0)），使用当前日期
            if latest_volatility_date < pd.Timestamp("2000-01-01"):
                latest_volatility_date = pd.to_datetime(datetime.now())
            
            # 修复相关性矩阵日期处理
            correlations = self.calculate_correlation()
            
            # 1. 获取最新的日期键并转换
            latest_date_str = list(correlations.keys())[-1]
            
            # 2. 安全转换日期格式
            try:
                # 处理整数索引（如果是位置索引而非日期索引）
                if isinstance(latest_date_str, int):
                    # 从收益率数据中获取对应日期
                    if len(self.returns_data.index) > latest_date_str:
                        latest_date = pd.to_datetime(self.returns_data.index[latest_date_str])
                    else:
                        latest_date = pd.to_datetime(self.returns_data.index[-1])
                else:
                    # 处理字符串日期
                    latest_date = pd.to_datetime(latest_date_str)
            except:
                # 转换失败时使用合并数据中的最新日期
                latest_date = pd.to_datetime(self.merged_data['date'].iloc[-1])
            
            # 3. 再次检查日期有效性
            if latest_date < pd.Timestamp("2000-01-01"):
                latest_date = pd.to_datetime(datetime.now())
            
            latest_correlation = correlations[latest_date_str]
            
            # 生成HTML报告
            html_content = f"""
            <html>
            <head>
                <meta charset="utf-8">
                <title>大类资产配置分析报告 - {report_date}</title>
                <style>
                    body {{ font-family: Arial, sans-serif; margin: 30px; }}
                    h1, h2 {{ color: #2c3e50; }}
                    .metric-table {{ border-collapse: collapse; width: 100%; margin: 20px 0; }}
                    .metric-table th, .metric-table td {{ border: 1px solid #ddd; padding: 12px; text-align: left; }}
                    .metric-table th {{ background-color: #f8f9fa; }}
                    .chart-container {{ margin: 30px 0; }}
                    .highlight {{ color: #e74c3c; font-weight: bold; }}
                </style>
            </head>
            <body>
                <h1>大类资产配置分析报告</h1>
                <p>报告日期: {datetime.now().strftime("%Y年%m月%d日")}</p>
                
                <h2>1. 资产绩效指标</h2>
                <table class="metric-table">
                    <tr>
                        <th>资产类别</th>
                        <th>总收益率</th>
                        <th>年化收益率</th>
                        <th>年化波动率</th>
                        <th>夏普比率</th>
                        <th>最大回撤</th>
                    </tr>
            """
            
            # 添加绩效指标到HTML
            for asset, metrics in performance_metrics.iterrows():
                html_content += f"""
                    <tr>
                        <td>{asset}</td>
                        <td>{metrics['总收益率']:.2%}</td>
                        <td>{metrics['年化收益率']:.2%}</td>
                        <td>{metrics['年化波动率']:.2%}</td>
                        <td>{metrics['夏普比率']:.2f}</td>
                        <td>{metrics['最大回撤']:.2%}</td>
                    </tr>
                """
            
            # 生成图表
            self._plot_charts()
            
            # 继续添加HTML内容，使用修复后的日期变量
            html_content += f"""
                </table>
                
                <h2>2. 最新波动率 ({latest_volatility_date.strftime("%Y年%m月%d日")})</h2>
                <div class="chart-container">
                    <img src="volatility_chart.png" alt="波动率图表" width="800">
                </div>
                
                <h2>3. 资产相关性矩阵（最近60个交易日，截至{latest_date.strftime("%Y年%m月%d日")}）</h2>
                <div class="chart-container">
                    <img src="correlation_matrix.png" alt="相关性矩阵" width="800">
                </div>
                
                <h2>4. 资产价格走势</h2>
                <div class="chart-container">
                    <img src="price_trend.png" alt="资产价格走势" width="800">
                </div>
                
                <h2>5. 资产收益率走势</h2>
                <div class="chart-container">
                    <img src="return_trend.png" alt="资产收益率走势" width="800">
                </div>
                
                <h2>6. 配置建议</h2>
                <p>基于最新数据分析，提出以下配置建议：</p>
                <ul>
                    <li>波动率最高的资产: {latest_volatility.idxmax()} ({latest_volatility.max():.2%})</li>
                    <li>夏普比率最高的资产: {performance_metrics['夏普比率'].idxmax()} ({performance_metrics['夏普比率'].max():.2f})</li>
                    <li>相关性最高的资产对: {self._find_highest_correlation(latest_correlation)}</li>
                    <li>相关性最低的资产对: {self._find_lowest_correlation(latest_correlation)}</li>
                </ul>
            </body>
            </html>
            """
            
            # 保存HTML报告
            with open(report_filename, "w", encoding="utf-8") as f:
                f.write(html_content)
            
            print(f"报告已生成: {report_filename}")
            return report_filename
    
    # 在_plot_charts方法中，修改相关性矩阵图表的标题部分
    def _plot_charts(self):
        """生成各类各类分析图表"""
        import matplotlib.pyplot as plt
        plt.rcParams["font.family"] = ["FangSong"]
        sns.set(font_scale=1.2)
        sns.set_style("whitegrid")

        # 1. 波动率图表
        volatility = self.calculate_volatility(window=60)
        plt.figure(figsize=(12, 8))
        for asset in volatility.columns:
            # 将索引转换为datetime格式
            dates = pd.to_datetime(volatility.index)
            plt.plot(dates, volatility[asset], label=asset)
        plt.title("各类资产60日滚动波动率（年化）")
        plt.ylabel("波动率")
        plt.xlabel("日期")
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout()
        plt.savefig(f"{self.result_dir}/volatility_chart.png")
        plt.close()
        
        # 2. 相关性矩阵 - 修复日期格式问题
        correlations = self.calculate_correlation(window=60)
        # 获取最近的日期并转换为datetime对象
        latest_date_str = list(correlations.keys())[-1]
        latest_date = pd.to_datetime(latest_date_str)
        
        latest_correlation = correlations[latest_date_str]
        plt.figure(figsize=(12, 10))
        sns.heatmap(latest_correlation, annot=True, cmap="coolwarm", vmin=-1, vmax=1)
        # 使用转换后的datetime对象格式化日期
        plt.title(f"资产相关性矩阵（最近60个交易日，截至{latest_date.strftime('%Y-%m-%d')}）")
        plt.tight_layout()
        plt.savefig(f"{self.result_dir}/correlation_matrix.png")
        plt.close()
        
        # 3. 资产价格走势（归一化处理）- 同样修复日期格式
        price_data = self.merged_data.set_index("date")
        normalized_prices = price_data / price_data.iloc[0] * 100
        plt.figure(figsize=(12, 8))
        for asset in normalized_prices.columns:
            # 将日期字符串转换为datetime
            dates = pd.to_datetime(normalized_prices.index)
            plt.plot(dates, normalized_prices[asset], label=asset)
        plt.title("各类资产价格走势（归一化，初始值=100）")
        plt.ylabel("价格指数（初始=100）")
        plt.xlabel("日期")
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout()
        plt.savefig(f"{self.result_dir}/price_trend.png")
        plt.close()
        
        # 4. 资产收益率走势 - 修复日期格式
        plt.figure(figsize=(12, 8))
        for asset in self.returns_data.columns:
            # 将日期字符串转换为datetime
            dates = pd.to_datetime(self.returns_data.index)
            plt.plot(dates, (1 + self.returns_data[asset]).cumprod() - 1, label=asset)
        plt.title("各类资产累计收益率")
        plt.ylabel("累计收益率")
        plt.xlabel("日期")
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout()
        plt.savefig(f"{self.result_dir}/return_trend.png")
        plt.close()
    
    def _find_highest_correlation(self, correlation_matrix):
        """找到相关性最高的资产对"""
        # 排除对角线元素
        mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
        triu_df = correlation_matrix.mask(mask)
        
        # 找到最大值
        max_corr = triu_df.stack().max()
        max_pair = triu_df.stack().idxmax()
        
        return f"{max_pair[0]} 和 {max_pair[1]}（{max_corr:.2f}）"
    
    def _find_lowest_correlation(self, correlation_matrix):
        """找到相关性最低的资产对"""
        # 排除对角线元素
        mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
        triu_df = correlation_matrix.mask(mask)
        
        # 找到最小值
        min_corr = triu_df.stack().min()
        min_pair = triu_df.stack().idxmin()
        
        return f"{min_pair[0]} 和 {min_pair[1]}（{min_corr:.2f}）"
    
    def send_report_email(self, report_filename, recipient_email, sender_email, sender_password, smtp_server="smtp.qq.com", smtp_port=465):
        """发送报告邮件"""
        # 创建邮件
        msg = MIMEMultipart()
        msg['From'] = sender_email
        msg['To'] = recipient_email
        msg['Subject'] = f"大类资产配置分析报告 - {datetime.now().strftime('%Y%m%d')}"
        
        # 邮件正文
        body = f"您好，\n\n附件是最新的大类资产配置分析报告，包含各类资产的收益率、波动率和相关性分析。\n\n报告日期：{datetime.now().strftime('%Y年%m月%d日')}"
        msg.attach(MIMEText(body, 'plain'))
        
        # 添加附件
        with open(report_filename, "rb") as attachment:
            part = MIMEBase('application', 'octet-stream')
            part.set_payload(attachment.read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', f"attachment; filename= {os.path.basename(report_filename)}")
        msg.attach(part)
        
        # 添加图片附件
        for img in ["volatility_chart.png", "correlation_matrix.png", "price_trend.png", "return_trend.png"]:
            img_path = f"{self.result_dir}/{img}"
            if os.path.exists(img_path):
                with open(img_path, "rb") as attachment:
                    part = MIMEBase('image', 'png')
                    part.set_payload(attachment.read())
                encoders.encode_base64(part)
                part.add_header('Content-Disposition', f"attachment; filename= {img}")
                msg.attach(part)
        
        # 发送邮件
        try:
            server = smtplib.SMTP_SSL(smtp_server, smtp_port)
            server.login(sender_email, sender_password)
            server.sendmail(sender_email, recipient_email, msg.as_string())
            server.quit()
            print(f"报告已发送至 {recipient_email}")
            return True
        except Exception as e:
            print(f"邮件发送失败: {str(e)}")
            return False

if __name__ == "__main__":
    # 创建跟踪器实例
    tracker = AssetAllocationTracker()
    
    # 获取过去3年的资产数据
    tracker.fetch_asset_data(days=365*3)
    
    # 生成分析报告
    report_file = tracker.generate_report()
    
    # 发送邮件（请替换为实际的邮箱信息）
    # 注意：使用QQ邮箱需要开启SMTP服务并获取授权码
    # sender_email = "your_email@qq.com"
    # sender_password = "your_email_password"  # 对于QQ邮箱，这是授权码而非登录密码
    # recipient_email = "recipient@example.com"
    
    # 发送报告（实际使用时取消注释）
    # tracker.send_report_email(report_file, recipient_email, sender_email, sender_password)


正在获取从 20220908 到 20250907 的资产数据...
成功获取 hs300 数据，共 726 条记录
成功获取 zz500 数据，共 726 条记录
成功获取 spx500 数据，共 751 条记录
成功获取 composite_bond_index 数据，共 749 条记录
成功获取 comodity_gold 数据，共 727 条记录
数据合并完成，共 801 个交易日
收益率计算完成，共 800 个有效数据点


C:\Users\admin\AppData\Local\Temp\ipykernel_3516\2976076118.py:124: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_df = merged_df.fillna(method="ffill").dropna()
C:\Users\admin\AppData\Local\Temp\ipykernel_3516\2976076118.py:372: UserWarning: Glyph 26085 (\N{CJK UNIFIED IDEOGRAPH-65E5}) missing from font(s) Arial.
  plt.tight_layout()
C:\Users\admin\AppData\Local\Temp\ipykernel_3516\2976076118.py:372: UserWarning: Glyph 26399 (\N{CJK UNIFIED IDEOGRAPH-671F}) missing from font(s) Arial.
  plt.tight_layout()
C:\Users\admin\AppData\Local\Temp\ipykernel_3516\2976076118.py:372: UserWarning: Glyph 27874 (\N{CJK UNIFIED IDEOGRAPH-6CE2}) missing from font(s) Arial.
  plt.tight_layout()
C:\Users\admin\AppData\Local\Temp\ipykernel_3516\2976076118.py:372: UserWarning: Glyph 21160 (\N{CJK UNIFIED IDEOGRAPH-52A8}) missing from font(s) Arial.
  plt.tight_layout()
C:\Users\admin\AppData\Local\Temp\ipyker

报告已生成: asset_allocation_reports/资产配置分析报告_20250907.html


In [14]:
from matplotlib.font_manager import FontManager
import subprocess

mpl_fonts = set(f.name for f in FontManager().ttflist)

print('all font list get from matplotlib.font_manager:')
for f in sorted(mpl_fonts):
    print('\t' + f)

all font list get from matplotlib.font_manager:
	Arial
	Bahnschrift
	Calibri
	Cambria
	Candara
	Comic Sans MS
	Consolas
	Constantia
	Corbel
	Courier New
	DejaVu Sans
	DejaVu Sans Display
	DejaVu Sans Mono
	DejaVu Serif
	DejaVu Serif Display
	DengXian
	Ebrima
	FangSong
	Franklin Gothic Medium
	Gabriola
	Gadugi
	Georgia
	HYZhongHei
	Impact
	Ink Free
	Javanese Text
	KaiTi
	Leelawadee UI
	Lucida Console
	Lucida Sans Unicode
	MS Gothic
	MT Extra
	MV Boli
	Malgun Gothic
	Microsoft Himalaya
	Microsoft JhengHei
	Microsoft New Tai Lue
	Microsoft PhagsPa
	Microsoft Sans Serif
	Microsoft Tai Le
	Microsoft YaHei
	Microsoft Yi Baiti
	MingLiU-ExtB
	Mongolian Baiti
	Myanmar Text
	Nirmala UI
	Noto Sans SC
	Noto Serif SC
	Palatino Linotype
	STIXGeneral
	STIXNonUnicode
	STIXSizeFiveSym
	STIXSizeFourSym
	STIXSizeOneSym
	STIXSizeThreeSym
	STIXSizeTwoSym
	Sans Serif Collection
	Segoe Fluent Icons
	Segoe MDL2 Assets
	Segoe Print
	Segoe Script
	Segoe UI
	Segoe UI Emoji
	Segoe UI Historic
	Segoe UI Symbol
	Se

In [17]:
days=365*3
end_date = datetime.now().strftime("%Y%m%d")
start_date = (datetime.now() - timedelta(days=days)).strftime("%Y%m%d")
df = cached_Ashare_index_hist(symbol='000300', start_date=start_date,
                           end_date=end_date)
df

,日期,开盘,收盘,最高,最低,成交量,成交额,振幅,涨跌幅,涨跌额,换手率
0,2022-09-08,4059.17,4037.68,4067.76,4036.02,100886049,1.868890e+11,0.78,-0.43,-17.30,0.31
1,2022-09-09,4050.59,4093.79,4099.01,4042.25,118427114,2.214425e+11,1.41,1.39,56.11,0.36
2,2022-09-13,4112.01,4111.11,4128.11,4097.14,111789737,2.185211e+11,0.76,0.42,17.32,0.34
3,2022-09-14,4058.04,4065.36,4089.06,4047.92,90793681,1.854850e+11,1.00,-1.11,-45.75,0.28
4,2022-09-15,4078.65,4027.12,4088.81,3998.86,123951196,2.543675e+11,2.21,-0.94,-38.24,0.38
...,...,...,...,...,...,...,...,...,...,...,...
721,2025-09-01,4516.88,4523.71,4528.13,4487.53,320366708,7.703783e+11,0.90,0.60,26.95,0.98
722,2025-09-02,4521.98,4490.45,4548.89,4454.42,303932566,7.779993e+11,2.09,-0.74,-33.26,0.93
723,2025-09-03,4501.34,4459.83,4510.73,4427.52,254320217,6.556971e+11,1.85,-0.68,-30.62,0.78
724,2025-09-04,4461.25,4365.21,4465.50,4327.51,307934609,7.731246e+11,3.09,-2.12,-94.62,0.94


In [23]:
report_file

'asset_allocation_reports/资产配置分析报告_20250907.html'